# Create metrics_config.toml

In this notebook we create the metrics_config.toml file. This file contains the SQL queries we want to do on the Delft-FIAT output to extract the data needed for the infographics.

In [1]:
from pathlib import Path
import pandas as pd
from os import makedirs
import tomli_w

In [2]:
model_root = Path('c:/Repositories/DT-flood/FloodAdapt_database/Humber/')

scenario_name = 'empty_event_current_no_measures'

fiat_out_path = model_root / 'output' / 'scenarios' / scenario_name / 'Impacts' / ('Impacts_detailed_'+scenario_name+'.csv')

metrics_path = model_root / 'static' / 'templates' / 'infometrics' / 'metrics_config.toml'

In [3]:
fiat_out = pd.read_csv(fiat_out_path)

In [4]:
fiat_out

,Unnamed: 0.1,Object ID,geometry,Unnamed: 0,Primary Object Type,Secondary Object Type,Max Potential Damage: Total,Ground Floor Height,Extraction Method,Ground Elevation,Damage Function: Total,Aggregation Label: AdminZonesLevel3,Inundation Depth,Reduction Factor,Damage: Total,Total Damage
0,0,801694934,"POLYGON ((-0.6437754 53.7975316, -0.6437767346...",0,residential,residential,2.475505e-01,0,centroid,1.782204,residential_total,East Riding of Yorkshire,NaN,NaN,NaN,0.0
1,1,1011539405,"POLYGON ((-0.8559087 53.6178324, -0.8559902 53...",1,residential,residential,3.684712e+01,0,centroid,1.782204,residential_total,East Riding of Yorkshire,NaN,NaN,NaN,0.0
2,2,318837779,"POLYGON ((-0.6499097 53.779979, -0.64990118262...",2,residential,residential,4.230074e+01,0,centroid,14.064857,residential_total,East Riding of Yorkshire,NaN,NaN,NaN,0.0
3,3,318837900,"POLYGON ((-0.6497855 53.7796279, -0.6497583 53...",3,residential,residential,2.233418e+02,0,centroid,14.064857,residential_total,East Riding of Yorkshire,NaN,NaN,NaN,0.0
4,4,318838002,"POLYGON ((-0.6498182 53.7797244, -0.6497855 53...",4,residential,residential,8.927019e+02,0,centroid,14.064857,residential_total,East Riding of Yorkshire,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136008,136008,85439161,"POLYGON ((-0.6041184 53.5675652, -0.6022228 53...",136008,industrial,industrial,7.192024e+07,0,centroid,20.117226,industrial_total,North Lincolnshire,NaN,NaN,NaN,0.0
136009,136009,2053662,"POLYGON ((-0.6242466 53.5812444, -0.6242798 53...",136009,industrial,industrial,7.707120e+07,0,centroid,36.400448,industrial_total,North Lincolnshire,NaN,NaN,NaN,0.0
136010,136010,1090620843,"POLYGON ((-0.4069093 53.6871899, -0.4064015 53...",136010,industrial,industrial,7.692684e+07,0,centroid,9.139746,industrial_total,North Lincolnshire,NaN,NaN,NaN,0.0
136011,136011,258191164,"POLYGON ((-1.0325138 53.5419609, -1.032564 53....",136011,commercial,commercial,1.420200e+08,0,centroid,11.070409,commercial_total,Doncaster,NaN,NaN,NaN,0.0


In [5]:
print(fiat_out['Primary Object Type'].unique())
print(fiat_out['Secondary Object Type'].unique())

['residential' 'commercial' 'industrial']
['residential' 'commercial' 'industrial' 'retail']


## Querie

A querie needs to consist of the following:
- `name`: ID of the querie
- `description`: explanation of what to calculate
- `select`: SQL command as str how to calculate quantity (e.g. COUNT, SUM)
- `filter`: SQL command as str which data to select (e.g. which column)
- `long_name`
- `show_in_metrics_table`

In [6]:
querie_list = []

Automate metric generation more. Make widgets with dropdown menu storing values into parameters (one for building type, one for stat to be queried, threshold etc) and make function that takes these parameters in and generates the right SQL querie and block in toml file

In [7]:
querie_single = {
    "name": "ResidentialMajorCount",
    "description": "No. Residential buildings with major damage",
    "select": "SUM(*)",
    "filter": "`Primary Object Type` IN ('RES') AND `Inundation Depth` <= 6 AND `Inundation Depth` > 1",
    "long_name": "Major Damaged Residential Buildings",
    "show_in_metrics_table": 'True'
}


In [8]:
querie_list.append(querie_single)
print(querie_list)

[{'name': 'ResidentialMajorCount', 'description': 'No. Residential buildings with major damage', 'select': 'SUM(*)', 'filter': "`Primary Object Type` IN ('RES') AND `Inundation Depth` <= 6 AND `Inundation Depth` > 1", 'long_name': 'Major Damaged Residential Buildings', 'show_in_metrics_table': 'True'}]


## Aggregation

A list of some aggregation to be used. Any entry should have a corresponding entry in the site.toml file under fiat.aggregation

In [9]:
agg_list = []

## Write toml file

In [10]:
metric_dict = {
    "queries": querie_list,
}

if len(agg_list) > 0:
    metric_dict.update({"aggregateBy": agg_list})
    
print(metric_dict)

{'queries': [{'name': 'ResidentialMajorCount', 'description': 'No. Residential buildings with major damage', 'select': 'SUM(*)', 'filter': "`Primary Object Type` IN ('RES') AND `Inundation Depth` <= 6 AND `Inundation Depth` > 1", 'long_name': 'Major Damaged Residential Buildings', 'show_in_metrics_table': 'True'}]}


In [11]:
if not metrics_path.parent.exists():
    makedirs(metrics_path.parent)

with open(metrics_path,'wb') as f:
    tomli_w.dump(metric_dict,f)